In [1]:
"""
Created on Sat May 08 

Functions used to create Landsat 8 cloud and shadow free composites for given period and area of interest (aoi).

Code sources:
- https://blog.gishub.org/earth-engine-tutorial-32-machine-learning-with-earth-engine-supervised-classification
- https://geohackweek.github.io/GoogleEarthEngine/05-classify-imagery/
- https://ceholden.github.io/open-geo-tutorial/python/chapter_5_classification.html
- GEE Documentation

@author: Joycelyn Longdon

"""

'\nCreated on Sat May 08 \n\nFunctions used to create Landsat 8 cloud and shadow free composites for given period and area of interest (aoi).\n\nCode sources:\n- https://blog.gishub.org/earth-engine-tutorial-32-machine-learning-with-earth-engine-supervised-classification\n- https://geohackweek.github.io/GoogleEarthEngine/05-classify-imagery/\n- https://ceholden.github.io/open-geo-tutorial/python/chapter_5_classification.html\n- GEE Documentation\n\n@author: Joycelyn Longdon\n\n'

In [2]:
import ee
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os
import ipygee as ui
import datetime
import pandas as pd

#Trigger the authentication flow.
ee.Authenticate()

#Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g6UbYilEG5UH24nuABXMM_4a_Jr2WMDgaslTpOj6VmkTGXnNMnIukE

Successfully saved authorization token.


In [3]:
#initialise map
Map = geemap.Map()

### Define Variables

In [20]:
#AOI geometry
AOI_FILE_PATH = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/wwc_concesion.GeoJSON')

#define study period
start_date = '2013-01-01'
end_date = '2019-12-31'
#create an array of start dates beginning the first of every month between 2013 and 2019
#adapted from https://riptutorial.com/pandas/example/6438/create-a-sample-dataframe-with-datetime
START_DATE = pd.date_range('2013-01-01', periods=7, freq='MS')
END_DATE = pd.date_range('2013-01-01', periods=7, freq='M') #there are 84 months between 2013 and 2019
print(END_DATE)
#path to save image
OUTPUT_PATH = "/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/Output Data"

#visual parameters
VIZ_PARAMS = {'min': 0,'max': 3000,'bands': ['B4', 'B3', 'B2']} #RGB Composite 

#satellite data source
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

#directories to save outputs
#output_dir = '/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/Train'



DatetimeIndex(['2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30',
               '2013-05-31', '2013-06-30', '2013-07-31'],
              dtype='datetime64[ns]', freq='M')


### Define functions to process imagery

In [21]:
#define area of interest
def open_aoi_json(aoi_file_path):
    with open(aoi_file_path) as f:
        studyRegion = json.load(f)
    studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
    #print(aoi)
    return studyRegion

studyRegion = open_aoi_json(AOI_FILE_PATH)

In [22]:
#define cloud mask
def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [23]:
#reduce image and clip to aoi
def clip_image_to_aoi(image):
    #reduce image collection into one multiband image taking the median value of each pixel
    image = image.median()
    #clip image to required boundary for studyRegion
    return image.clip(aoi) #NEED TO FIGURE OUT HOW TO DO THIS SEAMLESSLY WITH THE VARYING AOI'S

In [24]:
"""
for start_date in START_DATE:
    for end_date in END_DATE:
        start_date = start_date.date()
        end_date = end_date.date()
        print(start_date.date, end_date.date)
"""


'\nfor start_date in START_DATE:\n    for end_date in END_DATE:\n        start_date = start_date.date()\n        end_date = end_date.date()\n        print(start_date.date, end_date.date)\n'

In [25]:

#create monthly Landsat composite 

def monthly_imagery(image, start_date, end_date):
            image = image.filterDate(str(start_date), str(end_date))
            cloud_free_image = image.map(cloudMask)#apply cloud mask
            median_image = cloud_free_image.median().select(['B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa'])#take median pixel values for all bands
            monthly_image = median_image.clip(studyRegion)#clip to studyRegion
            return monthly_image

start_date, end_date = '2013-01-01','2013-12-31'
l8_monthly_image = monthly_imagery(l8, start_date, end_date)
#print(l8_monthly_image)
#l8_monthly_image.getInfo()
Map.addLayer(l8_monthly_image, VIZ_PARAMS, 'L8')
Map

Map(bottom=66510.0, center=[-1.8508742952122357, 17.95200347900391], controls=(WidgetControl(options=['positio…

In [61]:
list_of_images = [] #create empty list

for start_date_, end_date_ in zip(START_DATE, END_DATE):
    if not isinstance(start_date_, str):
      start_date_ = start_date_.date()
    if not isinstance(end_date_, str):
      end_date_ = end_date_.date()
    image = monthly_imagery(l8, start_date_, end_date_)

    list_of_images.append(image)
    
#list_of_images[0]
list_of_images


In [63]:
'''
for img in list_of_images:
    date = ee.Date(img.get('system:time_start'))
    print('Timestamp: ', date.getInfo())
'''

"\nfor img in list_of_images:\n    date = ee.Date(img.get('system:time_start'))\n    print('Timestamp: ', date.getInfo())\n"

In [68]:
# Reduce the region in order to get statistics of pixel values. The region parameter is the Feature geometry.
meanDictionary = list_of_images[6].reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': studyRegion,
  'scale': 30,
  'maxPixels': 1e10
})

# The result is a Dictionary.  Print it.
print(meanDictionary.getInfo())

{'B10': 2942.6643205071887, 'B11': 2917.841829042203, 'B2': 293.7126582165831, 'B3': 502.8471463382246, 'B4': 356.2441411348906, 'B5': 2189.714605647221, 'B6': 1267.2830024604077, 'B7': 548.6656129613929, 'pixel_qa': 322.00404200126167}


### Data Download
Code adapted from: https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb


In [12]:
#download each image as numpy array
#for i in list_of_images:
    #rgb_img_array = geemap.ee_to_numpy(list_of_images[i], region=studyRegion)
    #print(rgb_img_array.shape)
    # Scale the data to [0, 255] to show as an RGB image. 
    # Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
    #rgb_img_test = (255*((rgb_img_array[:, :, 0:3] - 100)/3500)).astype('uint8')
    #plt.imshow(rgb_img_test)
    #plt.show()
#download each image as GeoTiff
#and file naming convention

#input_name = 'main_ndombe_train'
#k = len(mai_ndombe_train)
#filename = '{}/{}_{}.tif'.format(output_dir, input_name, k)
#mai_ndombe_train_1 = mai_ndombe_train[1]
"""
task_config = {
    'region': studyRegion,
    'scale': 30,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'l8_monthly_image',
    'folder':'yearly_training_data',
    'image': l8_monthly_image,                          # changed this line
    'maxPixels': 1e9,
}
task=ee.Export.image.toDrive(**task_config)
task.start()
"""


"\ntask_config = {\n    'region': studyRegion,\n    'scale': 30,\n    'fileFormat': 'GeoTIFF',\n    'fileNamePrefix': 'l8_monthly_image',\n    'folder':'yearly_training_data',\n    'image': l8_monthly_image,                          # changed this line\n    'maxPixels': 1e9,\n}\ntask=ee.Export.image.toDrive(**task_config)\ntask.start()\n"

In [65]:

def download_tif(images=list_of_images):
    for img in images:
        #studyregion = 'wwc_concession_test'
        #k = len(list_of_images)
        #image_name = '{}/{}.tif'.format(studyregion,k)
        task_config = {
            'region': studyRegion,
            'scale': 30,
            'fileFormat': 'GeoTIFF',
            'fileNamePrefix': 'wwc_concession',
            'folder':'wwc_concession_test_data',
            'image': img,
            'crs':'EPSG:4326', #coordinate system that matches landcover data later on
            'maxPixels': 1e10,
        }
        task=ee.batch.Export.image.toDrive(**task_config)
        task.start()

download_tif(list_of_images)      


#def download_tif(image):
    #for image in list_of_images:
        #geemap.ee_export_image(image, filename=filename, scale=30, region=studyRegion, file_per_band=False)
        
#download_tif(l8_monthly_image)    
        

### Mapping

In [ ]:
#def display_image_and_save(image, vis_params, layer_name):
    #tif file
    #geemap.ee_export_image(landcover, 'Land_Cover_Classification.tif', scale=900)
    #Map.addLayer(image, vis_params, layer_name)
    #Map



### Save and visualise imagery

In [ ]:
'''
#retrieve monthly landsat imagery for region of interest and years
def yearly_collection(y):
    yearCollection = image.filter(ee.Filter.calendarRange(y,y,'year'))
    return yearCollection

def monthly_collection(m):
    monthCollection = yearCollection.filter(ee.Filter.calendarRange(m,m,'month'))
    return monthCollection
    
def monthly_image(image):
    date = (y, m, 1).format("MM_dd_YYYY")
    #filter imagery to study region and study period
    image = image.filter(ee.Filter.And(ee.Filter.geometry(studyRegion)))
    return monthly_image

l8_monthly = monthly_image(l8)
'''

In [ ]:
"""
#javascript version of what I need, I can't decode it

var startDate = ee.Date('2013-01-01'); // set start time for analysis
var endDate = ee.Date('2017-12-31'); // set end time for analysis

// calculate the number of months to process
var nMonths = ee.Number(endDate.difference(startDate,'month')).round();

var point = ee.Geometry.Point([-87.02617187499999, 28.05714582901274]);
var sst = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select('sst')
            .filterDate(startDate, endDate);

var byMonth = ee.ImageCollection(
  // map over each month
  ee.List.sequence(0,nMonths).map(function (n) {
    // calculate the offset from startDate
    var ini = startDate.advance(n,'month');
    // advance just one month
    var end = ini.advance(1,'month');
    // filter and reduce
    return sst.filterDate(ini,end)
                .select(0).mean()
                .set('system:time_start', ini);
}));

print(byMonth);

Map.addLayer(ee.Image(byMonth.first()),{min: 15, max: 35},'SST');
"""

In [ ]:
"""
#create monthly Landsat composite 

def monthly_imagery(image):
            image = image.filterDate(start_date, end_date)
            date_filtered = image.filter(ee.Filter.calendarRange(1,12,'month'))
            cloud_free_image = date_filtered.map(cloudMask)#apply cloud mask
            median_image = cloud_free_image.median().select(['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa'])#take median pixel values for all bands
            monthly_image = median_image.clip(studyRegion)#clip to studyRegion
            return monthly_image

l8_monthly_images = monthly_imagery(l8)
#l8_monthly_images.getInfo()
print(l8_monthly_images)
"""